In [128]:
!pip install nltk

In [129]:
import pandas as pd
import numpy as np
import json
import ast
from nltk.tokenize import WhitespaceTokenizer
import random
from azureml.core import Workspace, Dataset

In [130]:
subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'CougsInAzure'
workspace_name = 'CougsInAzure'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Alpha_Featurized')

In [131]:
df = dataset.to_pandas_dataframe()
df.head()

,LowerUnderscoreUpper,HasUnderscore,LowerUpperLower,MultipleLowerUpperLower,ExactlyTwoUppercase,AllLowerMoreThan,AdjacentUppers,StartLetterEndNonLetter,LengthGTLT,IllegalHostnameChars,...,ContainsPeriod,HasSlash,HasMultipleSlash,HasPossibleExtension,NumbersThenPeriod,AtLeastFourDigits,HasPeriodAndSlash,HasInstanceNumPeriod,HasMultipleNumPeriod,Label
0,False,False,False,False,True,True,True,False,True,True,...,True,False,False,False,True,True,False,True,True,1
1,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,False,False,False,True,False,False,True,True,...,True,True,False,True,False,False,False,False,False,1
3,False,False,False,False,False,True,False,True,True,True,...,False,False,False,False,False,True,False,False,False,0
4,False,False,False,False,False,True,False,False,True,True,...,True,True,False,True,False,False,False,False,False,1


In [134]:
# splitting the datasets into positive and negative lists
def IsPositive(r):
    return True if r[-1] else False
def IsNegative(r):
    return True if not r[-1] else False

df_list = df.values.tolist()


positive_dataset = list(filter(IsPositive, df_list))
negative_dataset = list(filter(IsNegative, df_list))

random.shuffle(positive_dataset)
random.shuffle(negative_dataset)



28552
[[False, False, False, False, False, True, False, False, True, True, False, False, True, True, False, True, False, False, False, False, False, 1], [False, False, False, False, False, True, False, False, True, True, False, False, True, True, False, True, False, False, False, True, True, 1], [False, False, True, False, True, False, True, True, True, True, True, True, True, False, False, False, False, True, False, False, False, 1], [False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, 1], [False, False, False, False, False, True, False, False, False, True, False, False, True, True, True, True, False, True, False, True, True, 1], [False, True, True, False, True, True, True, True, True, True, True, True, True, False, False, False, False, True, False, False, False, 1], [False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False

In [135]:
# Combining the dataset into 1
dataset = []
l = len(negative_dataset) if len(negative_dataset) < len(positive_dataset) else len(positive_dataset)

for i in range(0,l):
    dataset.append(positive_dataset[i])
    dataset.append(negative_dataset[i])
print(dataset[:5])
print(len(dataset))

[[False, False, False, False, False, True, False, False, True, True, False, False, True, True, False, True, False, False, False, False, False, 1], [False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, 0], [False, False, False, False, False, True, False, False, True, True, False, False, True, True, False, True, False, False, False, True, True, 1], [False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, 0], [False, False, True, False, True, False, True, True, True, True, True, True, True, False, False, False, False, True, False, False, False, 1]]
25220


In [137]:
# Splitting the dataset into train and test

X = list(map(lambda x: x[:-1], dataset))
y = list(map(lambda x: x[-1], dataset))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
print(len(X))

25220


In [31]:
# Functions to compute different metrics

def calcAccuracy(target, predictions):
    total = len(predictions)
    correct = 0
    for i in range(len(predictions)):
        if target[i] == predictions[i]:
            correct += 1
        accuracy = float(correct) / float(total)
    return accuracy

def FalsePositives(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 0 and pred[i] == 1:
            false_positives.append(x_test[i])
    return false_positives

def FalseNegatives(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] == 1 and pred[i] == 0:
            false_negatives.append(x_test[i])
    return false_negatives

def PrintResults(classifier, acc, recall, precision):
    print(classifier + " Classifier")
    print("Accuracy= ", acc)
    print("Precision= ", precision)
    print("Recall= ", recall)

In [30]:
# BEGIN MACHINE LEARNING MODELS
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [138]:
# DECISION TREE CLASSIFIER
dt = DecisionTreeClassifier().fit(X_train, y_train)
pred = dt.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Decision Tree", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

Decision Tree Classifier
Accuracy=  0.9312747807281029
Precision=  0.929526994744386
Recall=  0.9335412667946257

FP-> 295 
FN-> 277


In [119]:
# KNN CLASSIFIER
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("KNN", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

KNN Classifier
Accuracy=  0.9186591373302896
Precision=  0.9227415839186244
Recall=  0.9141074856046065

FP-> 319 
FN-> 358


In [120]:
# LOGISTIC REGRESSION CLASSIFER
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Logistic Regression", acc, recall, precision)


Logistic Regression Classifier
Accuracy=  0.8942688934278505
Precision=  0.9091587854654056
Recall=  0.8764395393474088


In [121]:
# RANDOM FORREST
rfm = RandomForestClassifier().fit(X_train, y_train)
pred = rfm.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Random Forrest", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

Random Forrest Classifier
Accuracy=  0.9288718010332813
Precision=  0.9275466284074605
Recall=  0.930662188099808

FP-> 303 
FN-> 289


In [122]:
# NAIVE BAYS
nb = GaussianNB().fit(X_train, y_train)
pred = nb.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Naive Bays", acc, recall, precision)

Naive Bays Classifier
Accuracy=  0.7156073531178662
Precision=  0.932724651609803
Recall=  0.46569097888675626


In [58]:
# Try/Catch for recognizing if input is json or not

#data_dict = list(map(lambda x : ast.literal_eval(x), data))
for x in dataset[:5]:
    print(x)

for y in dataset:
    x = None
    try:
        x = ast.literal_eval(y[0])
        #print(f"It didn't break! {x}")
    except (ValueError, SyntaxError):
        z = True
        #print(f"It broke:( {y}") 



['#ReTweetThis for #OurMischief #BBlogRT SGH_RTs GFXCoach FearRTs ShoutGamers Demented_RTs Mighty_RTs DynoRTs NightRetweets WitWGARA posted: https://t.co/oAcPcjMyvAWatch Ninja now streaming VALORANTat https://t.co/QghgQSwvrL- #ThankYou for being #OurMischief! Wit', 1]
['TRUDYDELEON-', 0]
['groucho-eu', 1]
['7JERRY-NELSON', 0]
['RT tkasasagi: My Kuzushiji recognition mobile app using Tensorflow and Flutter project got awarded research grant from Japan Science and T', 1]


In [59]:
tk = WhitespaceTokenizer() 
t1 = "Hello"
token = tk.tokenize(t1)
print(token)

['Hello']
